In [1]:
#!python.exe -m pip install --upgrade pip
# !pip install imparaai-checkers

In [2]:
# !pip install -U scikit-learn

In [3]:
from checkers.game import Game
import copy
import random
from sklearn.ensemble import RandomForestRegressor
import time
import numpy as np  # Import NumPy

In [4]:
class Heuristics:
    @staticmethod
    def get_player_pieces(game, player):
        player_pieces = []
        for piece in game.board.pieces:
            if piece.player == player and not piece.captured:
                player_pieces.append(piece)
        return player_pieces
    
    @staticmethod
    def get_opponent_pieces(game, player):
        opponent_pieces = []
        for piece in game.board.pieces:
            if piece.player != player and not piece.captured:
                opponent_pieces.append(piece)
        return opponent_pieces
    
    #heuristics list starts here
    #increase the number of pieces of the player as much as possible, maximize the difference between the player and opponent
    @staticmethod
    def simple_piece_count(game, player):
        player_pieces = Heuristics.get_player_pieces(game, player)
        opponent_pieces = Heuristics.get_opponent_pieces(game, player)
        
        return len(player_pieces) - len(opponent_pieces)
    
    #try to increase the number of king pieces the player has
    @staticmethod
    def king_count(game, player):
        player_pieces = Heuristics.get_player_pieces(game, player)
        return sum(1 for piece in player_pieces if piece.king)
    
    #try to have as much as dominance possible in the centre board
    @staticmethod
    def evaluate_board_control(game, player):
        #Higher score for central squares, Medium score for middle squares, lower score for back rows
        board_control_scores = {
            (1, 2, 3, 4, 5, 6, 7, 8): 5,  
            (9, 10, 11, 12, 21, 22, 23, 24): 3,  
            (13, 14, 15, 16, 17, 18, 19, 20): 1, 
        }

        total_score = 0
        # Use a set to collect unique positions
        total_positions = set()  
        
        pieces = Heuristics.get_player_pieces(game, player)
        
        for piece in pieces:
            for positions, score in board_control_scores.items():
                if piece.position in positions:
                    total_score += score
                    # Add the positions in the group to the set
                    total_positions.update(positions)
                    # Assign the highest available score  
                    break  

        return total_score
    #heuristics list ends here    

In [5]:
class MinimaxAgent(Heuristics):
    def __init__(self, player):
        self.player = player
        self.regressor = None
    
    def handle_empty_data(self, game, heuristic, observations):
        valid_moves = game.get_possible_moves()
        # Implement your logic for handling empty data here
        if not valid_moves:
            return None, False
        return random.choice(valid_moves), True

    def get_best_move(self, game, heuristic, observations, data, stage):
        valid_moves = game.get_possible_moves()  # Get a list of valid moves

        if stage == "RF":
            # Try to use the regressor if it's available
            if self.regressor:
                _, best_move = self.minimax(game, 3, True, float('-inf'), float('inf'), heuristic, 2, observations)
                if best_move in valid_moves:
                    return best_move, False
                else:
                    return random.choice(valid_moves), True

        _, best_move = self.minimax(game, 3, True, float('-inf'), float('inf'), heuristic, 2, observations, stage)
        
        # Ensure the best move is a valid move
        if best_move in valid_moves:
            return best_move, False
        if stage == "RF" or stage == "Hl":
            # If all else fails, handle empty data
            move, flag = self.handle_empty_data(game, heuristic, observations)
            return move, flag



    def minimax(self, game, depth, maximizing_player, alpha, beta, heuristic, l, observations, stage):
        #selection of the heuristics for position evaluation when either the game is over or at the leaf nodes
        if depth == 0 or game.is_over():
            if stage == "H0" or stage == "alpha-beta":
                return self.evaluate_H0(game, heuristic), None
            elif stage == "Hl":
                return self.evaluate_Hl(game, heuristic, l), None
            elif stage == "RF":
                if self.regressor:
                    predicted_HL = self.regressor.predict([list(observations.values())])
                    return predicted_HL[0], None
                else:
                    # If the regressor is not trained, fall back to H0
                    return self.evaluate_H0(game, heuristic), None
            
        if maximizing_player:
            # Initialize the maximum evaluation.
            max_eval = float('-inf')
            # Initialize the best move.
            best_move = None
            
            # Loop through possible moves in the game.
            for move in game.get_possible_moves():
                # Create a copy of the game for simulation.
                next_game = copy.deepcopy(game)
                # Apply the current move to the copy of the game.
                next_game.move(move)

                if stage == "H0":
                    eval = self.evaluate_H0(next_game, heuristic)
                elif stage == "Hl" or stage == "RF" or stage == "alpha-beta":
                    eval, _ = self.minimax(next_game, depth - 1, False, alpha, beta, heuristic, l, observations, stage)
                
    
                # If the evaluation is better than the current maximum: update the maximum evaluationand best move
                if eval > max_eval:
                    max_eval = eval
                    best_move = move

                # Update the alpha value.
                alpha = max(alpha, max_eval)
                # Prune the search if the beta value is less than or equal to alpha.
                if beta <= alpha:
                    break

            return max_eval, best_move
        else:
            # Initialization of the minimizing player
            min_eval = float('inf')
            best_move = None

            # Loop through possible moves in the game.    
            for move in game.get_possible_moves():
                # Create a copy of the game for simulation.
                next_game = copy.deepcopy(game)
                next_game.move(move)

                if stage == "H0":
                    eval = self.evaluate_H0(next_game, heuristic)
                elif stage == "Hl" or stage == "RF" or stage == "alpha-beta":
                    eval, _ = self.minimax(next_game, depth - 1, True, alpha, beta, heuristic, l, observations, stage)
                
                # If the evaluation is worse than the current minimum: update the minimum evaluation and best move
                if eval < min_eval:
                    min_eval = eval
                    best_move = move    

                # Update the alpha value.
                beta = min(beta, min_eval)
                # Prune the search if the beta value is less than or equal to alpha.
                if beta <= alpha:
                    break

            return min_eval, best_move
        
    @staticmethod
    def preprocess_data(data):
        X = [list(observation.values()) for observation, hl_value in data]
        y = [hl_value for observation, hl_value in data]
        return X, y

    def train_regressor(self, data):
        X, y = MinimaxAgent.preprocess_data(data)
        print("Number of samples in X before filtering:", len(X))
        print("Number of samples in y before filtering:", len(y))

        # Filter out samples with non-finite y values and corresponding X values
        finite_indices = [i for i, value in enumerate(y) if np.isfinite(value)]
        if len(finite_indices) != len(y):
            print(f"Removing {len(y) - len(finite_indices)} samples with non-finite y values.")

        X = [X[i] for i in finite_indices]
        y = [y[i] for i in finite_indices]

        print("Number of samples in X after filtering:", len(X))
        print("Number of samples in y after filtering:", len(y))

        # Ensure that X and y have the same number of samples
        if len(X) != len(y):
            raise ValueError("Number of samples in X and y do not match")

        regressor = RandomForestRegressor()
        if len(X) > 0:
            regressor.fit(X, y)
        self.regressor = regressor
    
    
    # Add a method to calculate H0 evaluation for a given game state
    def evaluate_H0(self, game, heuristic):
        if heuristic == "piece_count":
            return Heuristics.simple_piece_count(game, self.player)
        elif heuristic == "king_count":
            return Heuristics.king_count(game, self.player)
        else:
            return Heuristics.evaluate_board_control(game, self.player)
        
    def evaluate_Hl(self, game, heuristic, l):
        if l == 0:
            return self.evaluate_H0(game, heuristic)

        def depth_l_evaluation(game, current_depth):
            if current_depth == l:
                return self.evaluate_H0(game, heuristic)

            maximizing_player = (current_depth % 2 == 0)

            if maximizing_player:
                max_eval = float('-inf')
                for move in game.get_possible_moves():
                    next_game = copy.deepcopy(game)
                    if next_game.move(move):
                        eval = depth_l_evaluation(next_game, current_depth + 1)
                        max_eval = max(max_eval, eval)
                return max_eval
            else:
                min_eval = float('inf')
                for move in game.get_possible_moves():
                    next_game = copy.deepcopy(game)
                    if next_game.move(move):
                        eval = depth_l_evaluation(next_game, current_depth + 1)
                        min_eval = min(min_eval, eval)
                return min_eval

        # Initialize the evaluation to be computed at level l.
        return depth_l_evaluation(game, current_depth=0)





In [6]:
def play_game(agent, game, heuristic, data, stage):
    current_player = game.whose_turn()
    game.consecutive_noncapture_move_limit = 20
    opponent_moves_count = 0
    agent_moves_count = 0
    observations = {}

    while not game.is_over():
        if current_player == agent.player:
            if stage == "RF":
                observations = {
                    'num_pieces': len(agent.get_player_pieces(game, agent.player))
                }
                
            best_move, random_checker = agent.get_best_move(game, heuristic, observations, data, stage)
            # Check if the best_move is valid
            valid_moves = game.get_possible_moves()
            
            if best_move in valid_moves:
                game.move(best_move)  # Apply the move
                agent_moves_count += 1
                
                if stage == "RF":
                    # Collect high-level value for this move (hl_value)
                    hl_value = agent.evaluate_Hl(game, heuristic, 2)

                    # Append observations and hl_value as a tuple to the data list
                    data.append((observations, hl_value))
                # print(f"Your agent's move: {best_move}")
                
                if not random_checker:
                    1
                    # print(f"Your agent's move: {best_move}")
                else:
                    2
                    # print(f"Your agent's random move: {best_move} + {random_checker}")
                    
        else:
            #the opponent plays randomly for baseline comparison
            opponent_moves = game.get_possible_moves()
            opponent_move = random.choice(opponent_moves)
            game.move(opponent_move)
            opponent_moves_count += 1
            # print(f"Opponent's move: {opponent_move}")
            
        # Switch the current player for the next turn
        if not game.move_limit_reached():
            current_player = game.whose_turn()
        else:
            break
    
    piece_count_agent_moves.append(agent_moves_count)
    # print(f"Total moves by your agent: {agent_moves_count}")
    # print(f"Total moves by the opponent: {opponent_moves_count}")
    winner = game.get_winner()

    if winner == 1:
        # print("The agent won!")
        return 1
    elif winner == 2:
        # print("The opponent won!")
        return -1
    else:
        # print("It's a draw!")
        return 0


piece_count_wins = []
piece_count_losses = []
piece_count_draws = []
piece_count_durations = []
piece_count_agent_moves = []
data = []
stages = ["alpha-beta" ,"H0", "Hl", "RF"]
# stages = ["Hl"]
for i in range(10):
    for stage in stages:
        heuristic = 'piece_count'
        print(f"Comparing {heuristic} for maximizing agents for iteration-{i} and {stage} evaluation")
        start_time = time.time()
        game = Game()
        agent = MinimaxAgent(1)
        result = play_game(agent, game, heuristic, data, stage)
        piece_count_durations.append(time.time() - start_time)

        if result == 1:
            piece_count_wins.append(1)
            piece_count_losses.append(0)
            piece_count_draws.append(0)
        elif result == -1:
            piece_count_wins.append(0)
            piece_count_losses.append(1)
            piece_count_draws.append(0)
        else:
            piece_count_wins.append(0)
            piece_count_losses.append(0)
            piece_count_draws.append(1)

# Train the regressor using the collected data
if stage == "RF":
    agent.train_regressor(data)

# print(f"piece_count_wins {piece_count_wins}")
# print(f"piece_count_losses {piece_count_losses}")
# print(f"piece_count_draws {piece_count_draws}")
print(f"piece_count_durations {piece_count_durations}")
# print(f"piece_count_moves {piece_count_agent_moves}")

Comparing piece_count for maximizing agents for iteration-0 and alpha-beta evaluation
Comparing piece_count for maximizing agents for iteration-0 and H0 evaluation
Comparing piece_count for maximizing agents for iteration-0 and Hl evaluation
Comparing piece_count for maximizing agents for iteration-0 and RF evaluation
Comparing piece_count for maximizing agents for iteration-1 and alpha-beta evaluation
Comparing piece_count for maximizing agents for iteration-1 and H0 evaluation
Comparing piece_count for maximizing agents for iteration-1 and Hl evaluation
Comparing piece_count for maximizing agents for iteration-1 and RF evaluation
Comparing piece_count for maximizing agents for iteration-2 and alpha-beta evaluation
Comparing piece_count for maximizing agents for iteration-2 and H0 evaluation
Comparing piece_count for maximizing agents for iteration-2 and Hl evaluation
Comparing piece_count for maximizing agents for iteration-2 and RF evaluation
Comparing piece_count for maximizing age

In [7]:
# from checkers.game import Game
# import copy
# import random
# from sklearn.ensemble import RandomForestRegressor
# import time
# import numpy as np  # Import NumPy


# class Heuristics:
#     @staticmethod
#     def get_player_pieces(game, player):
#         player_pieces = []
#         for piece in game.board.pieces:
#             if piece.player == player and not piece.captured:
#                 player_pieces.append(piece)
#         return player_pieces
    
#     @staticmethod
#     def get_opponent_pieces(game, player):
#         opponent_pieces = []
#         for piece in game.board.pieces:
#             if piece.player != player and not piece.captured:
#                 opponent_pieces.append(piece)
#         return opponent_pieces
    
#     @staticmethod
#     def simple_piece_count(game, player):
#         player_pieces = Heuristics.get_player_pieces(game, player)
#         opponent_pieces = Heuristics.get_opponent_pieces(game, player)
        
#         return len(player_pieces) - len(opponent_pieces)
    
#     @staticmethod
#     def king_count(game, player):
#         player_pieces = Heuristics.get_player_pieces(game, player)
#         return sum(1 for piece in player_pieces if piece.king)
    
#     @staticmethod
#     def evaluate_board_control(game, player):
#         board_control_scores = {
#             (1, 2, 3, 4, 5, 6, 7, 8): 5,  
#             (9, 10, 11, 12, 21, 22, 23, 24): 3,  
#             (13, 14, 15, 16, 17, 18, 19, 20): 1, 
#         }

#         total_score = 0
#         total_positions = set()  
        
#         pieces = Heuristics.get_player_pieces(game, player)
        
#         for piece in pieces:
#             for positions, score in board_control_scores.items():
#                 if piece.position in positions:
#                     total_score += score
#                     total_positions.update(positions)
#                     break  

#         return total_score

# class MinimaxAgent(Heuristics):
#     def __init__(self, player):
#         self.player = player
#         self.regressor = None

#     def handle_empty_data(self, game, heuristic, observations):
#         valid_moves = game.get_possible_moves()
#         # Implement your logic for handling empty data here
#         if not valid_moves:
#             return None, False
#         return random.choice(valid_moves), True

#     def get_best_move(self, game, heuristic, observations, data):
#         valid_moves = game.get_possible_moves()
#         if not valid_moves:
#             return None, False

#         # Try to use the regressor if it's available
#         if self.regressor:
#             _, best_move = self.minimax(game, 3, True, float('-inf'), float('inf'), heuristic, 2, observations)
#             if best_move in valid_moves:
#                 return best_move, False
#             else:
#                 return random.choice(valid_moves), True

#         # If no regressor, then try to use the minimax logic to get a move
#         _, best_move = self.minimax(game, 3, True, float('-inf'), float('inf'), heuristic, 2, observations)
#         if best_move in valid_moves:
#             return best_move, False

#         # If all else fails, handle empty data
#         move, flag = self.handle_empty_data(game, heuristic, observations)
#         return move, flag


#     def minimax(self, game, depth, maximizing_player, alpha, beta, heuristic, l, observations):
#         if depth == 0 or game.is_over():
#             if self.regressor:
#                 predicted_HL = self.regressor.predict([list(observations.values())])
#                 return predicted_HL[0], None
#             else:
#                 # If the regressor is not trained, fall back to H0
#                 return self.evaluate_H0(game), None


#         if maximizing_player:
#             max_eval = float('-inf')
#             best_move = None

#             for move in game.get_possible_moves():
#                 next_game = copy.deepcopy(game)
#                 next_game.move(move)
#                 eval, _ = self.minimax(next_game, depth - 1, False, alpha, beta, heuristic, l, observations)

#                 if eval > max_eval:
#                     max_eval = eval
#                     best_move = move

#                 alpha = max(alpha, max_eval)
#                 if beta <= alpha:
#                     break

#             return max_eval, best_move
#         else:
#             min_eval = float('inf')
#             best_move = None

#             for move in game.get_possible_moves():
#                 next_game = copy.deepcopy(game)
#                 next_game.move(move)
#                 eval, _ = self.minimax(next_game, depth - 1, True, alpha, beta, heuristic, l, observations)

#                 if eval < min_eval:
#                     min_eval = eval
#                     best_move = move

#                 beta = min(beta, min_eval)
#                 if beta <= alpha:
#                     break

#             return min_eval, best_move

#     @staticmethod
#     def preprocess_data(data):
#         X = [list(observation.values()) for observation, hl_value in data]
#         y = [hl_value for observation, hl_value in data]
#         return X, y

#     def train_regressor(self, data):
#         X, y = MinimaxAgent.preprocess_data(data)
#         print("Number of samples in X before filtering:", len(X))
#         print("Number of samples in y before filtering:", len(y))

#         # Filter out samples with non-finite y values and corresponding X values
#         finite_indices = [i for i, value in enumerate(y) if np.isfinite(value)]
#         if len(finite_indices) != len(y):
#             print(f"Removing {len(y) - len(finite_indices)} samples with non-finite y values.")

#         X = [X[i] for i in finite_indices]
#         y = [y[i] for i in finite_indices]

#         print("Number of samples in X after filtering:", len(X))
#         print("Number of samples in y after filtering:", len(y))

#         # Ensure that X and y have the same number of samples
#         if len(X) != len(y):
#             raise ValueError("Number of samples in X and y do not match")

#         regressor = RandomForestRegressor()
#         if len(X) > 0:
#             regressor.fit(X, y)
#         self.regressor = regressor


#     def evaluate_H0(self, game):
#         # Getting multiple observations
#         piece_count = Heuristics.simple_piece_count(game, self.player)
#         # king_count = Heuristics.king_count(game, self.player)
#         # board_control = Heuristics.evaluate_board_control(game, self.player)

#         # You can combine these observations into a composite score in various ways. 
#         # Here, I'm just combining them linearly as an example.
#         # In practice, you might want to give different weights or use a trained model.
#         # return piece_count + king_count + board_control
#         return piece_count


#     def evaluate_Hl(self, game, heuristic, l):
#         if l == 0:
#             return self.evaluate_H0(game)

#         def depth_l_evaluation(game, current_depth):
#             if current_depth == l:
#                 return self.evaluate_H0(game)

#             maximizing_player = (current_depth % 2 == 0)

#             if maximizing_player:
#                 max_eval = float('-inf')
#                 for move in game.get_possible_moves():
#                     next_game = copy.deepcopy(game)
#                     if next_game.move(move):
#                         eval = depth_l_evaluation(next_game, current_depth + 1)
#                         max_eval = max(max_eval, eval)
#                 return max_eval
#             else:
#                 min_eval = float('inf')
#                 for move in game.get_possible_moves():
#                     next_game = copy.deepcopy(game)
#                     if next_game.move(move):
#                         eval = depth_l_evaluation(next_game, current_depth + 1)
#                         min_eval = min(min_eval, eval)
#                 return min_eval

#         return depth_l_evaluation(game, current_depth=0)

# def play_game(agent, game, heuristic, data):
#     current_player = game.whose_turn()
#     game.consecutive_noncapture_move_limit = 20
#     opponent_moves_count = 0
#     agent_moves_count = 0
#     while not game.is_over():
#         if current_player == agent.player:
#             observations = {
#                 'num_pieces': len(agent.get_player_pieces(game, agent.player))
#             }
#             valid_moves = game.get_possible_moves()
#             best_move, random_checker = agent.get_best_move(game, heuristic, observations, data)

#             if best_move in valid_moves:
#                 game.move(best_move)
#                 agent_moves_count += 1  # Update the agent's move count

#                 # Collect high-level value for this move (hl_value)
#                 hl_value = agent.evaluate_Hl(game, heuristic, 2)

#                 # Append observations and hl_value as a tuple to the data list
#                 data.append((observations, hl_value))
#                 if not random_checker:
#                     print(f"Your agent's move: {best_move}")
#                 else:
#                     print(f"Your agent's random move: {best_move} and {random_checker}")
#             else:
#                 print("Invalid move selected by the agent")
#         else:
#             opponent_moves = game.get_possible_moves()
#             opponent_move = random.choice(opponent_moves)
#             game.move(opponent_move)
#             opponent_moves_count += 1
#             print(f"Opponent's move: {opponent_move}")

#         if not game.move_limit_reached():
#             current_player = game.whose_turn()
#         else:
#             break

#     piece_count_agent_moves.append(agent_moves_count)  # Update the agent's move count
#     print(f"Total moves by your agent: {agent_moves_count}")
#     print(f"Total moves by the opponent: {opponent_moves_count}")

#     winner = game.get_winner()

#     if winner == 1:
#         print("The agent won!")
#         return 1
#     elif winner == 2:
#         print("The opponent won!")
#         return -1
#     else:
#         print("It's a draw!")
#         return 0


# piece_count_wins = []
# piece_count_losses = []
# piece_count_draws = []
# piece_count_durations = []
# piece_count_agent_moves = []
# data = []

# for i in range(10):
#     heuristic = 'piece_count'
#     print(f"Comparing {heuristic} for maximizing agents for iteration-{i}")
#     start_time = time.time()
#     game = Game()
#     agent = MinimaxAgent(1)
#     result = play_game(agent, game, heuristic, data)
#     piece_count_durations.append(time.time() - start_time)

#     if result == 1:
#         piece_count_wins.append(1)
#         piece_count_losses.append(0)
#         piece_count_draws.append(0)
#     elif result == -1:
#         piece_count_wins.append(0)
#         piece_count_losses.append(1)
#         piece_count_draws.append(0)
#     else:
#         piece_count_wins.append(0)
#         piece_count_losses.append(0)
#         piece_count_draws.append(1)

# # Train the regressor using the collected data
# agent.train_regressor(data)

# print(f"piece_count_wins {piece_count_wins}")
# print(f"piece_count_losses {piece_count_losses}")
# print(f"piece_count_draws {piece_count_draws}")
# print(f"piece_count_durations {piece_count_durations}")
# print(f"piece_count_moves {piece_count_agent_moves}")


In [8]:
# import matplotlib.pyplot as plt
# import numpy as np

# def game_outcome(wins, losses, draws, heuristic_name, iterations_per_heuristic):
#     iterations = []

#     for i in range(iterations_per_heuristic):
#         iterations.append("Iteration-" + str(i))
    
#     win_percentage = [win / iterations_per_heuristic * 100 for win in wins]
#     lose_percentage = [loss / iterations_per_heuristic * 100 for loss in losses]
#     draw_percentage = [draw / iterations_per_heuristic * 100 for draw in draws]
    
#     # Create an array of the x positions
#     x = np.arange(len(iterations))
#     bar_width = 0.25
    
#     # Plot the results as a grouped bar chart
#     plt.figure(figsize=(10, 6))
#     plt.bar(x - bar_width, win_percentage, bar_width, label='Win', color='green', alpha=0.6)
#     plt.bar(x, lose_percentage, bar_width, label='Lose', color='red', alpha=0.6)
#     plt.bar(x + bar_width, draw_percentage, bar_width, label='Draw', color='blue', alpha=0.6)

#     plt.xlabel('Iterations')
#     plt.ylabel('Percentage')
#     plt.title('Game Outcomes Analysis for ' + heuristic_name)
#     plt.xticks(x, iterations)
#     plt.legend(loc='upper right')
#     plt.show()



In [9]:
# game_outcome(piece_count_wins, piece_count_losses, piece_count_draws, "Piece Count", 10)

In [10]:
# game_outcome(king_count_wins, king_count_losses, king_count_draws, "King Count", 10)

In [11]:
# game_outcome(board_control_wins, board_control_losses, board_control_draws, "Board Control", 10)

In [12]:
# piece_count_average_duration = sum(piece_count_durations) / len(piece_count_durations)
# king_count_average_duration = sum(king_count_durations) / len(king_count_durations)
# board_control_average_duration = sum(board_control_durations) / len(board_control_durations)

# average_durations = [piece_count_average_duration, king_count_average_duration, board_control_average_duration]

# heuristics_list = ['Piece Count', 'King Count', 'Board Control']

# # Create a bar chart
# plt.bar(heuristics_list, average_durations, color=['red', 'green', 'blue'])
# plt.xlabel('Heuristic')
# plt.ylabel('Average Durations (seconds)')
# plt.title('Average Game Duration Analysis')

# # Display the plot
# plt.show()

In [13]:
# piece_count_average_agent_moves = sum(piece_count_agent_moves) / len(piece_count_agent_moves)
# king_count_average_agent_moves = sum(king_count_agent_moves) / len(king_count_agent_moves)
# board_control_average_agent_moves = sum(board_control_agent_moves) / len(board_control_agent_moves)

# average_agent_moves = [piece_count_average_agent_moves, king_count_average_agent_moves, board_control_average_agent_moves]

# # heuristics_list = ['Piece Count', 'King Count', 'Board Control']

# # Create a bar chart
# plt.bar(heuristics_list, average_agent_moves, color=['red', 'green', 'blue'])
# plt.xlabel('Heuristics')
# plt.ylabel('Average Agent Moves')
# plt.title('Average Agent Moves Analysis')

# # Display the plot
# plt.show()